# SEA Airport Imagery Retrieval


In [1]:
%config IPCompleter.greedy=True
import folium
import os
from zipfile import ZipFile
import numpy as np
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt 
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from shapely.geometry import MultiPolygon, Polygon
import rasterio as rio
from rasterio.plot import show
import rasterio.mask
import fiona
import pyproj
import gdal
from datetime import datetime
import pygc
from io import StringIO
import shutil
import cv2
from sklearn import cluster
from osgeo import gdal, gdal_array
from sklearn.cluster import KMeans
from PIL import Image
import scipy.misc
import json

In [2]:
missing_values = [r"\N"]
df0 = pd.read_csv(r'airport-codes_csv.csv',na_values = missing_values)
list1,list2,a,b = [],[],[],[]
for itema,itemb in zip(df0["iata_code"],df0['iso_region']):
    list1.append(itema)
    list2.append(itemb)
    
for items in df0['coordinates']:
    a.append(items.split(',')[0])
    b.append(items.split(',')[1])

In [3]:
iatalist,iatabatch= [],[]
batch_ranges = []
batch_size = 400
batch_number = 1
missing_values = [r"\N"]
#df = pd.read_excel(r'190812_CompanyRelations.xlsx', sheet_name = 'Airport Data',na_values = missing_values)
df = pd.read_csv(r'seaAirportDB.csv')
df_all = df
#.sort_values(by = ['2018 Passengers'], ascending = False)
#t = np.invert(df['2018 Passengers'].isnull())
#df = df[t]
#df = df.sort_values(by = ["2018 Passengers"], ascending = False) 
allNames = []
for items,names in zip(df_all['IATA'],df_all['Name']):
    iatabatch.append(items)
    allNames.append(names)
    
def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

iatabatch = Remove(iatabatch)
chunk_items = len(iatabatch) - 1

for i in range(chunk_items):
    batch_ranges.append(iatabatch[i*batch_size : (i+1)*batch_size])
    
batch_ranges

for i in range(batch_size):
    try:
        iatalist.append(batch_ranges[batch_number -1][i])
    except:
        continue
iatalist,allNames[:1000]
air = dict(zip(iatalist,allNames[:1000]))

In [4]:
iatalist

['CGK',
 'SIN',
 'KUL',
 'MNL',
 'SGN',
 'HAN',
 'DPS',
 'SUB',
 'UPG',
 'DAD',
 'CEB',
 'KNO',
 'JOG',
 'BKI',
 'BPN',
 'HLP',
 'PEN',
 'BTH',
 'RGN',
 'PLM',
 'KCH',
 'SRG',
 'CXR',
 'DVO',
 'PNK',
 'LOP',
 'PKU',
 'PDG',
 'BDO',
 'BDJ',
 'REP',
 'PNH',
 'PQC',
 'JHB',
 'SZB',
 'MDC',
 'LGK',
 'TKG',
 'SOC',
 'KLO',
 'DJJ',
 'MYY',
 'PGK',
 'VII',
 'KBR',
 'KOE',
 'ILO',
 'DJB',
 'CGY',
 'HUI',
 'PPS',
 'HPH',
 'SOQ',
 'BCD',
 'KDI',
 'DLI',
 'AMQ',
 'CRK',
 'SBW',
 'PLW',
 'MLG',
 'MPH',
 'BWN',
 'TWU',
 'BMV',
 'BTJ',
 'ZAM',
 'BKS',
 'TJQ',
 'TRK',
 'PKY',
 'TAG',
 'TTE',
 'TGG',
 'SDK',
 'GES',
 'AOR',
 'BTU',
 'BXU',
 'MKW',
 'GTO',
 'VCL',
 'PKN',
 'LGP',
 'MDL',
 'THD',
 'USU',
 'VCA',
 'LBJ',
 'SMQ',
 'MKQ',
 'KTG',
 'UIH',
 'WMX',
 'BMU',
 'TNJ',
 'BIK',
 'PXU',
 'OZC',
 'GNS',
 'RXS',
 'IPH',
 'VDH',
 'DPL',
 'KUA',
 'BUW',
 'ENE',
 'TUG',
 'PAG',
 'VCS',
 'LAO',
 'KOS',
 'WNP',
 'IAO',
 'BSO',
 'CYZ',
 'TWT',
 'SRI',
 'LBX',
 'DIN',
 'MKZ',
 'TBB',
 'MZV',
 'SUG',
 'LMN',


We have a database file of 6000+ airports containing their Latitude,Longitude,IATA codes etc. We use the consolidated list of interested we generated in the last block to query for the same from our Airports.dat file and prepare a new list of airport codes.

In [5]:
statefinderDict = dict(zip(list1,list2))

In [6]:
stateNames = []
for names in iatalist:
    try:
        stateNames.append(statefinderDict[str(names)])
    except: 
        continue

In [7]:
statefinderDict

{nan: 'JP-46',
 'UTK': 'MH-UTI',
 'OCA': 'US-FL',
 'PQS': 'US-AK',
 'CSE': 'US-CO',
 'JCY': 'US-TX',
 'PMX': 'US-MA',
 'WLR': 'US-AK',
 'NUP': 'US-AK',
 'PTC': 'US-AK',
 'ICY': 'US-AK',
 'PPV': 'US-AK',
 'KKK': 'US-AK',
 'MHS': 'US-CA',
 'NIR': 'US-TX',
 'GCT': 'US-AZ',
 'ELW': 'US-AK',
 'LVD': 'US-AK',
 'HGZ': 'US-AK',
 'OTN': 'US-IN',
 'TLF': 'US-AK',
 'BZT': 'US-TX',
 'HBH': 'US-AK',
 'FAK': 'US-AK',
 'BYW': 'US-WA',
 'DRF': 'US-AK',
 'BDF': 'US-IL',
 'VRS': 'US-MO',
 'GDH': 'US-AK',
 'ATT': 'US-AK',
 'LIV': 'US-AK',
 'PDB': 'US-AK',
 'KOZ': 'US-AK',
 'TNK': 'US-AK',
 'EVA': 'US-TX',
 'WHD': 'US-AK',
 'TKI': 'US-AK',
 'WKK': 'US-AK',
 'WFB': 'US-AK',
 'NNK': 'US-AK',
 'BKF': 'US-AK',
 'BCS': 'US-LA',
 'SPQ': 'US-CA',
 'TSS': 'US-NY',
 'QNY': 'US-NY',
 'BWL': 'US-OK',
 'WYB': 'US-AK',
 'CWS': 'US-WA',
 'TEK': 'US-AK',
 'DUF': 'US-NC',
 'SSW': 'US-WA',
 'FOB': 'US-CA',
 'PUL': 'US-WA',
 'CCD': 'US-CA',
 'WMK': 'US-AK',
 'AXB': 'US-NY',
 'REE': 'US-TX',
 'WDN': 'US-WA',
 'WWP': 'US-AK'

In [8]:
class Airport:    
    def __init__(self,name,city,con,iata,icao,lat,lon,alt):
        self.name = name
        self.city = city
        self.country = con
        self.iata = iata.upper() #3 letter
        self.icao = icao.upper() #4 letter
        self.lat = float(lat)
        self.lon = float(lon)
        self.alt = float(alt)
        
    def __str__(self):
        return ( str(self.iata+':'+self.name +':'+self.country))
    def __repr__(self):
        return ( str(self.iata+':'+self.name +':'+self.country))

airportPath = "airports.dat"

codes = {}
apts = []
with open(airportPath, encoding = "utf8") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        apt = Airport(row[1],row[2],row[3],
                      row[4],row[5],
                      float(row[6]),float(row[7]),float(row[8]))
        codes[row[4].upper()] = apt
        apts.append(apt)
        
#print("found %s airports"%(len(apts)))
X = codes["MEL"].lon
Y = codes["MEL"].lat
iata,iatalat,iatalon,notthere = [],[],[],[]
for key in iatalist:
    if key in list1:
        iata.append(key)
        #have changed codes --> list1 & codes[key].lat/lon --> a/b[list1.index(key)]
        iatalon.append(a[list1.index(key)])
        iatalat.append(b[list1.index(key)])
    else : 
        notthere.append(key)
    

#print(iata[0],iatalat[0],iatalon[0])


len(iatalat)

#printing the entries that we have in our excel but not in our database(significantly improved results over the .dat file previously supplied)
notthere

['SRI', 'VGN']

Using Python graphic library, we try to virtually construct a geo-boundary translated from our required distance (typically around 10-20kms). 

In [9]:
def latLonBoxByWandH(lat,lon,ew_width,ns_height):
    lats, lons = [], []
    #distance in m, az (in deg), lat (in deg), long (in deg)

    res = pygc.great_circle(distance=ew_width/2, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']

    res = pygc.great_circle(distance=ns_height/2, azimuth=180, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=270, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ns_height, azimuth=0, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)

    res = pygc.great_circle(distance=ew_width, azimuth=90, latitude=lat, longitude=lon)
    lat, lon = res['latitude'], res['longitude']
    lats.append(lat), lons.append(lon)
    
    return {'lats':lats,'lons':lons}


#test the function
#lengths in m
ew_width = 10000
ns_height = 10000
loc_lat,loc_lon = [], []
#loc_lat = [51.4775, 51.148056,1.359167]
#loc_lon = [-0.461389, -0.190278, 103.989444]
for i in range(0,len(iata)):
    loc_lat.append(iatalat[i])
    loc_lon.append(iatalon[i])
    #print(iata[i])

#loc_lon,loc_lat
loc_lat = np.asfarray(loc_lat,float)
loc_lon = np.asfarray(loc_lon,float)
len(loc_lat),len(loc_lon)

(142, 142)

Once we construct the spatial boundary, we then try to project in on a basemap of the globe and use Folium to try and do a sanity check if our geo-boundaries actually fall under the area of our concern, in this case airports.

In [10]:
polygons,footprint =[], []

for lat, lon in zip(loc_lat,loc_lon):

    box = latLonBoxByWandH(lat,lon,ew_width,ns_height)
    
    polygon_geom = Polygon(zip(box['lons'], box['lats']))
    footprint.append(polygon_geom)
    crs = {'init': 'epsg:4326'}
    polygon = gpd.GeoDataFrame(index=[0], crs=crs, geometry=[polygon_geom])       
    
    polygons.append(polygon)


pyproj.Proj("+init=epsg:4326")
m = folium.Map([loc_lat[0],loc_lon[0]], zoom_start=12)
for polygon in polygons:
    folium.GeoJson(polygon).add_to(m)
m


C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:280: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(projparams)


Making a query to Sentinel API using our boundary and specifying key parameters such as intersection, cloud cover, timeline of photo and the band of satellite.

In [11]:
user = 'demi12395' 
password = 'Sutd1234' 

api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')

In [12]:
apiq = [] 
for i in range(0,len(polygons)):
    apiq.append(api.query(footprint[i],
                 date = ('20210330','20201231'),
                 platformname = 'Sentinel-2',
                 processinglevel = 'Level-2A',
                area_relation = ('Contains'),
                 cloudcoverpercentage = (0,15)))
for i in range(len(apiq)):
    print(iatalist[i],len(apiq[i]))

SentinelAPIError: HTTP status 429 Too Many Requests: 
# 429 Too Many Requests

You have sent too many requests in a given amount of time.

Once we get a list of products (images in sentinel terminology), we try to work with the database and retrieve a consolidated list of product ID and title ID wherein the former is used as an unique identifier (to make another download query) in sentinel databse and latter gets stored as a folder in our local drive.

In [114]:
products_list, products_list_sorted,images,title,titlelist,best,bestlist =[],[], [], [], [] , [],[]
for products in apiq:
        products_list.append(api.to_geodataframe(products))
        
#Sorting the list of products within our array of locations for minimum cloudcover
for products in products_list:
    products_list_sorted.append(products.sort_values(['summary'],ascending = [False]))
    

for i in range(0,len(products_list_sorted)):
        images.append(products_list_sorted[i].head(1))

#for products in images: 
        #title.append(products.title)
        #best.append(products.uuid)
        
for items in title: 
        items[18:][:99999]
for products in images:
    for i in range(0,len(products)):
        title.append(products.title[i])
        best.append(products.uuid[i])
    bestlist.append(best)
    titlelist.append(title)
    title,best = [], []
        
#titlelist
#bestlist

C:\Anaconda3\envs\fastai_v1\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


Now, we are trying to retrieve the datestamp of our product from the database and storing it seperately for our records.

In [115]:
datestamp,stamp = [],[]
for k in range(0,len(bestlist)):
    for i in range(0,len(bestlist[k])):
        d = products_list[k].beginposition[i].date().strftime("%Y%m%d")
        stamp.append(d)
    datestamp.append(stamp)
    stamp = []
#products_list[1]
#datestamp

In [116]:
#kth best image available
k = 0

In [117]:
prodlist = bestlist
#prodlist

In [118]:
titlelist

[['S2A_MSIL2A_20200604T163901_N0214_R126_T16TDR_20200604T210710'],
 ['S2A_MSIL2A_20200518T200901_N0214_R028_T08VPH_20200518T235928'],
 ['S2A_MSIL2A_20200524T153601_N0214_R111_T19TEM_20200524T194324'],
 ['S2B_MSIL2A_20200415T190909_N0214_R056_T10UDU_20200415T233431'],
 ['S2A_MSIL2A_20200603T170901_N0214_R112_T15TWN_20200603T231006'],
 ['S2A_MSIL2A_20200527T153911_N0214_R011_T18TYP_20200527T215633'],
 ['S2A_MSIL2A_20200602T155911_N0214_R097_T17SNB_20200602T202645'],
 ['S2A_MSIL2A_20200527T204021_N0214_R014_T07VEG_20200528T004953'],
 ['S2A_MSIL2A_20200512T162901_N0214_R083_T16RBV_20200512T205014'],
 ['S2B_MSIL2A_20200415T190909_N0214_R056_T10UDU_20200415T233431'],
 ['S2B_MSIL2A_20200519T221529_N0214_R115_T04WES_20200520T005038'],
 ['S2A_MSIL2A_20200513T155911_N0214_R097_T17SPC_20200513T203041'],
 ['S2A_MSIL2A_20200526T160911_N0214_R140_T17SND_20200526T203535'],
 ['S2A_MSIL2A_20200526T160911_N0214_R140_T17SND_20200526T203535'],
 ['S2B_MSIL2A_20200519T153559_N0214_R111_T19TCG_20200519T21151

Now we are trying to make an API call to download our product. Note that we are adding a method to extract the product and remove the parent file just to conservr memory as each product could take upto 1 GB of storage space.

In [119]:
directory  = "Products"
counter = 1
for prods,titles in zip(prodlist,titlelist):
    for items,items2 in zip(prods,titles):
        print("Downloading product " + items2 + " ... " + str(counter) + "/" + str(len(titlelist)))
        counter+=1
        api.download(items, directory_path = directory)
        with ZipFile("Products/" + items2 + '.zip', 'r') as zf:
            print("extracting  " + items2)
            zf.extractall('Products/')
            zf.close()  
        print("Unzipping complete.. Now removing the file..")
        os.remove("Products/" + items2 + '.zip')

Downloading: 100%|████████████████████████████████████████████████████████████████| 1.06G/1.06G [03:13<00:00, 5.48MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.06G/1.06G [00:03<00:00, 344MB/s]


extracting  S2A_MSIL2A_20200604T163901_N0214_R126_T16TDR_20200604T210710
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.03G/1.03G [03:08<00:00, 5.47MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.03G/1.03G [00:03<00:00, 307MB/s]


extracting  S2A_MSIL2A_20200518T200901_N0214_R028_T08VPH_20200518T235928
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.16G/1.16G [04:36<00:00, 4.20MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.16G/1.16G [00:03<00:00, 347MB/s]


...

extracting  S2A_MSIL2A_20200603T170901_N0214_R112_T15STD_20200604T005104
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.17G/1.17G [03:23<00:00, 5.74MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.17G/1.17G [00:03<00:00, 346MB/s]


extracting  S2A_MSIL2A_20200530T154911_N0214_R054_T18SVJ_20200530T201834
Unzipping complete.. Now removing the file..


Downloading: 100%|████████████████████████████████████████████████████████████████| 1.14G/1.14G [03:33<00:00, 5.36MB/s]
MD5 checksumming: 100%|████████████████████████████████████████████████████████████| 1.14G/1.14G [00:03<00:00, 361MB/s]


extracting  S2B_MSIL2A_20200523T182919_N0214_R027_T11SMS_20200523T230158
Unzipping complete.. Now removing the file..


In [120]:
folder,foldername,dire,allmapsinc,allmaps,diretemp =[], [], [],[],[],[]
for titles in titlelist:
    for items in titles:
        d = os.listdir("Products/" + items + ".SAFE/" + "GRANULE")
        folder.append(d[0])
    foldername.append(folder)
    folder = []
foldername

for titles,folders in zip(titlelist,foldername):
    for items,names in zip(titles,folders):
        maindir = "Products/" + items + ".SAFE/" + "GRANULE/"  +str(names) + "/IMG_DATA" + "/R10m/" 
        allmapsinc.append(maindir)
        diretemp.append(os.listdir(maindir))
    allmaps.append(allmapsinc)
    dire.append(diretemp)
    allmapsinc,diretemp = [],[]

In [121]:
#EDITED BY SCJ
templatemp,template = [],[]
for k in range(0,len(dire)):
    for i in range(0,len(dire[0])):
        #templatemp.append(dire[k][i][-2][0:][:-4]) #SCJ removed as seems to rely on file order, doesn't work on mac
        files = [a for a in dire[k][i] if "_TCI_10m.jp2" in a]
        #print(files)
        templatemp.append(files[0][:-4])   
    template.append(templatemp)
    templatemp = []

In [122]:
#template

Dynamically retrieving the directory that we need for processing and rendering our image files.

In [123]:
allmapscompleteinc,allmapscomplete = [],[]
for k in range(0,len(prodlist)):
    for i in range(0,len(prodlist[0])):
        allmapscompleteinc.append(allmaps[k][i] + "/" + template[k][i] + ".jp2")
    allmapscomplete.append(allmapscompleteinc)
    allmapscompleteinc = []
#allmapscomplete

In [124]:
dataArr = []
for items in allmapscomplete:
    data = rio.open(items[0])
    #print(items)
    #print(data.meta['crs'])
    dataArr.append(data)
    data.close()

Opening a test image using Rasterio and checking its projection CRS and other metadata.

Plotting a sample image using matplotlib for a sanity check, as we are dealing with a huge data collection.

### Mask Satellite images

Now we try to reproject our images into EPSG system in order to average out for all the different parts of earth. This process could take a while.

In [125]:
from osgeo import gdal
filename,file = [],[]
#gdoptions = gdal.WarpOptions(options = ['dstSRS = "WGS84"','xRes = 10', "yRes = 10"," targetAlignedPixels = True"])
gdoptions = gdal.WarpOptions('dstSRS = WGS84')
for k in range(0,len(allmaps)):
    for i in range(0,len(allmaps[0])):
        input_raster = gdal.Open(allmapscomplete[k][i])
        output_raster = allmaps[k][i] + template[k][i] +".tiff"
        file.append(output_raster)
        options=['dstSRS = EPSG:4326','width = 10980', 'height = 10980']
        gdal.Warp(output_raster,input_raster,srcSRS = str(dataArr[k].meta['crs']),dstSRS = 'epsg:4326',width = str(dataArr[k].width),height = str(dataArr[k].height))
        print("Finished Warping for " + " " + str(k + 1) + " " + iatalist[k] )
    filename.append(file)
    file = []

Finished Warping for  1 IMT
Finished Warping for  2 WRG
Finished Warping for  3 PQI
Finished Warping for  4 FRD
Finished Warping for  5 HIB
Finished Warping for  6 LEB
Finished Warping for  7 LWB
Finished Warping for  8 YAK
Finished Warping for  9 PIB
Finished Warping for  10 CLM
Finished Warping for  11 GAL
Finished Warping for  12 SHD
Finished Warping for  13 MGW
Finished Warping for  14 CKB
Finished Warping for  15 PVC
Finished Warping for  16 MWA
Finished Warping for  17 APN
Finished Warping for  18 CEC
Finished Warping for  19 UIN
Finished Warping for  20 FOE
Finished Warping for  21 ILG
Finished Warping for  22 CLD


In [126]:
statefinderDict[str(iatalist[4])][3:]

'MN'

In [127]:
for items in filename:
    data = rio.open(items[0])
    #print(items)
    print(data.meta['crs'])
    data.close()

EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326
EPSG:4326


Nextly, we directly mask the image and write out the clipped photo in a seperate folder with our usual naming convention.

In [128]:
from rasterio.mask import mask
src_crs = []

geoms = []
for i in range(0,len(datestamp)):
    geoms.append(polygons[i]['geometry'])
    
for k in range(0,len(prodlist[0])):
    for i in range(0,len(prodlist)):
        with rasterio.open(filename[i][k]) as src:
                out_image, out_transform = mask(src, geoms[i], crop=True)
                out_meta = src.meta.copy()

        out_meta.update({"driver": "GTiff",
                             "height": out_image.shape[1],
                             "width": out_image.shape[2],
                             "transform": out_transform})
        try:
            with rasterio.open("USA_source/" + statefinderDict[str(iatalist[i])][3:] +  "_" +  iatalist[i] + "_" + datestamp[i][k]  +".tiff", "w", **out_meta) as dest:
                dest.write(out_image)
            #os.remove(titlelist[k][i])
            src.close()
            dest.close()
        except:
            continue
        

In [100]:
for items in os.listdir(directory):
    os.remove(directory + "/" + items)
    print("Removed " + items)

PermissionError: [WinError 5] Access is denied: 'Products/S2A_MSIL2A_20200513T155911_N0214_R097_T18TTK_20200513T203041.SAFE'

In [250]:
latlonfinderDict = dict(zip(iatalist,[[a,b,c] for a,b,c in zip(loc_lat,loc_lon,iatalist)]))

In [251]:
latlonfinderDict

{'ATL': [33.6367, -84.428101, 'ATL'],
 'LAX': [33.94250107, -118.4079971, 'LAX'],
 'ORD': [41.9786, -87.9048, 'ORD'],
 'DFW': [32.896801, -97.038002, 'DFW'],
 'DEN': [39.861698150635, -104.672996521, 'DEN'],
 'JFK': [40.63980103, -73.77890015, 'JFK'],
 'SFO': [37.61899948120117, -122.375, 'SFO'],
 'SEA': [47.449001, -122.308998, 'SEA'],
 'LAS': [36.08010101, -115.1520004, 'LAS'],
 'MCO': [28.429399490356445, -81.30899810791016, 'MCO'],
 'CLT': [35.2140007019043, -80.94309997558594, 'CLT'],
 'EWR': [40.692501068115234, -74.168701171875, 'EWR'],
 'MIA': [25.79319953918457, -80.29060363769531, 'MIA'],
 'PHX': [33.43429946899414, -112.01200103759766, 'PHX'],
 'IAH': [29.984399795532227, -95.34140014648438, 'IAH'],
 'BOS': [42.36429977, -71.00520325, 'BOS'],
 'MSP': [44.882, -93.221802, 'MSP'],
 'DTW': [42.212398529052734, -83.35340118408203, 'DTW'],
 'FLL': [26.072599, -80.152702, 'FLL'],
 'PHL': [39.87189865112305, -75.24109649658203, 'PHL'],
 'LGA': [40.77719879, -73.87259674, 'LGA'],
 '

In [252]:
latlonDict = {}
for airport in iatalist:
    try:
        stName = statefinderDict[str(airport)][3:]
        if stName not in latlonDict:
            latlonDict[str(stName)] = []
        latlonDict[str(stName)].append(latlonfinderDict[str(airport)])
    except:
        continue

In [254]:
latlonDict

{'GA': [[33.6367, -84.428101, 'ATL'],
  [32.12760162, -81.20210266, 'SAV'],
  [33.36989974975586, -81.9645004272461, 'AGS'],
  [32.516300201416016, -84.93890380859375, 'CSG'],
  [30.782499313354492, -83.27670288085938, 'VLD'],
  [31.258800506591797, -81.46649932861328, 'BQK'],
  [31.535499572753906, -84.19450378417969, 'ABY']],
 'CA': [[33.94250107, -118.4079971, 'LAX'],
  [37.61899948120117, -122.375, 'SFO'],
  [32.7336006165, -117.190002441, 'SAN'],
  [37.721298, -122.221001, 'OAK'],
  [37.362598, -121.929001, 'SJC'],
  [38.69540023803711, -121.59100341796875, 'SMF'],
  [33.67570114, -117.8679962, 'SNA'],
  [34.20069885253906, -118.35900115966797, 'BUR'],
  [34.055999755859375, -117.60099792480469, 'ONT'],
  [33.81769943, -118.1520004, 'LGB'],
  [33.8297004699707, -116.50700378417969, 'PSP'],
  [36.77619934082031, -119.71800231933594, 'FAT'],
  [34.42620087, -119.8399963, 'SBA'],
  [36.58700180053711, -121.84300231933594, 'MRY'],
  [38.50899887, -122.8130035, 'STS'],
  [35.2368011474

In [255]:
with open('latlonDict.json', 'w') as fp:
    json.dump(latlonDict, fp)

In [182]:
finalSet = os.listdir('USA_source')
finalSet
filepathDict = {}
for items in list2:
    stName = items[3:]
    for files in finalSet:
        if stName in files[:2]: 
            if stName not in filepathDict:
                filepathDict[stName] = []
            filepathDict[str(stName)].append(files)

KeyError: 'PA'

In [180]:
filepathDict['PA']

['PA_ABE_20200530.tiff',
 'PA_AVP_20200530.tiff',
 'PA_ERI_20200526.tiff',
 'PA_IPT_20200530.tiff',
 'PA_LBE_20200526.tiff',
 'PA_MDT_20200530.tiff',
 'PA_PHL_20200530.tiff',
 'PA_PIT_20200526.tiff',
 'PA_SCE_20200513.tiff',
 'PA_ABE_20200530.tiff',
 'PA_AVP_20200530.tiff',
 'PA_ERI_20200526.tiff',
 'PA_IPT_20200530.tiff',
 'PA_LBE_20200526.tiff',
 'PA_MDT_20200530.tiff',
 'PA_PHL_20200530.tiff',
 'PA_PIT_20200526.tiff',
 'PA_SCE_20200513.tiff',
 'PA_ABE_20200530.tiff',
 'PA_AVP_20200530.tiff',
 'PA_ERI_20200526.tiff',
 'PA_IPT_20200530.tiff',
 'PA_LBE_20200526.tiff',
 'PA_MDT_20200530.tiff',
 'PA_PHL_20200530.tiff',
 'PA_PIT_20200526.tiff',
 'PA_SCE_20200513.tiff',
 'PA_ABE_20200530.tiff',
 'PA_AVP_20200530.tiff',
 'PA_ERI_20200526.tiff',
 'PA_IPT_20200530.tiff',
 'PA_LBE_20200526.tiff',
 'PA_MDT_20200530.tiff',
 'PA_PHL_20200530.tiff',
 'PA_PIT_20200526.tiff',
 'PA_SCE_20200513.tiff',
 'PA_ABE_20200530.tiff',
 'PA_AVP_20200530.tiff',
 'PA_ERI_20200526.tiff',
 'PA_IPT_20200530.tiff',
